In [3]:
import sys
from pathlib import Path

sys.path.insert(0, "workspace/scrape")

import pandas as pd
from datetime import date
import re
import glob
from datetime import timedelta

from gspread_utils import init_gspread

pd.options.display.max_columns = None
pd.options.display.max_rows = 250

In [4]:
def phrase_drop(df, col, phrase):
    to_drop = df[col].str.contains(phrase, na=False).sum()
    print(f"Removing {to_drop} rows due to {phrase} in {col}. Rows remaining: {len(df) - to_drop}")
    return df[~df[col].str.contains(phrase, na=False)]

In [5]:
df = pd.read_csv("workspace/scrape/jobs/waymo/data/waymo_jobs_raw.csv")
df = phrase_drop(df, "Title", "PhD")
df = phrase_drop(df, "Title", "Principal")
df = phrase_drop(df, "Title", "Director")
df = phrase_drop(df, "Title", "Manager")
df = phrase_drop(df, "Title", "Android")
df = phrase_drop(df, "Title", "Senior Research Scientist")

Removing 2 rows due to PhD in Title. Rows remaining: 209
Removing 4 rows due to Principal in Title. Rows remaining: 205
Removing 5 rows due to Director in Title. Rows remaining: 200
Removing 16 rows due to Manager in Title. Rows remaining: 184
Removing 6 rows due to Android in Title. Rows remaining: 178
Removing 3 rows due to Senior Research Scientist in Title. Rows remaining: 175


In [6]:
# Split Title on first '-' OR ',' into two columns
_parts = df["Title"].astype(str).str.split(r"\s*[-,]\s*", n=1, expand=True)

# Avoid NaNs when a title has no '-' / ','
df["Role"] = _parts[0].fillna("").astype(str).str.strip() if _parts.shape[1] > 0 else ""
df["Team"] = _parts[1].fillna("").astype(str).str.strip() if _parts.shape[1] > 1 else ""
df = df.drop(columns=["Title"])

In [7]:
# Reorder columns: Role, Team, URL, Salary first; everything else after
first = [c for c in ["Role", "Team", "URL"] if c in df.columns]
rest = [c for c in df.columns if c not in first]
df = df[first + rest]

df.head(10)

,Role,Team,URL,Department,Location
0,Analysis Infra SWE,,https://careers.withwaymo.com/jobs/analysis-in...,SOFTWARE ENGINEERING,"MOUNTAIN VIEW, CALIFORNIA"
2,Data Engineer,,https://careers.withwaymo.com/jobs/data-engine...,SOFTWARE ENGINEERING,"MOUNTAIN VIEW, CALIFORNIA"
3,Data Scientist,,https://careers.withwaymo.com/jobs/data-scient...,SOFTWARE ENGINEERING,"MOUNTAIN VIEW, CALIFORNIA | SAN FRANCISCO, CAL..."
11,Fullstack Engineer,Waymo Applications and Tools,https://careers.withwaymo.com/jobs/fullstack-e...,SOFTWARE ENGINEERING,"MOUNTAIN VIEW, CALIFORNIA"
12,Infra SWE Query & Storage,,https://careers.withwaymo.com/jobs/infra-swe-q...,SOFTWARE ENGINEERING,"MOUNTAIN VIEW, CALIFORNIA"
13,Machine Learning Engineer,,https://careers.withwaymo.com/jobs/machine-lea...,SOFTWARE ENGINEERING,"MOUNTAIN VIEW, CALIFORNIA"
14,Machine Learning Engineer,Mapping,https://careers.withwaymo.com/jobs/machine-lea...,SOFTWARE ENGINEERING,"MOUNTAIN VIEW, CALIFORNIA"
15,Machine Learning Engineer,Mapping,https://careers.withwaymo.com/jobs/machine-lea...,SOFTWARE ENGINEERING,"MOUNTAIN VIEW, CALIFORNIA"
17,Machine Learning Engineer,ADV Systems,https://careers.withwaymo.com/jobs/machine-lea...,SOFTWARE ENGINEERING,"MOUNTAIN VIEW, CALIFORNIA | SAN FRANCISCO, CAL..."
18,Machine Learning Engineer,Data & Systems,https://careers.withwaymo.com/jobs/machine-lea...,SOFTWARE ENGINEERING,"MOUNTAIN VIEW, CALIFORNIA | SAN FRANCISCO, CAL..."


In [9]:
# Auth + open spreadsheet
gc = init_gspread()
sh = gc.open("Waymo Jobs")

# Find most recent date-named sheet that is not today
today = date.today()
date_re = re.compile(r"^\d{4}-\d{2}-\d{2}$")
candidates = []
for ws in sh.worksheets():
    if date_re.match(ws.title):
        sheet_date = date.fromisoformat(ws.title)
        if sheet_date < today:
            candidates.append((sheet_date, ws))

if not candidates:
    raise RuntimeError("No prior-day sheet found in 'Waymo Jobs'")

candidates.sort(reverse=True)
prev_date, prev_ws = candidates[0]
print(f"Most recent prior sheet: '{prev_ws.title}'")

df_prev = pd.DataFrame(prev_ws.get_all_records())

# Index both by URL
df_prev = df_prev.set_index("URL")
df = df.set_index("URL")

# Copy all columns from df_prev that don't already exist in df
for col in df_prev.columns:
    if col not in df.columns:
        df[col] = df_prev[col].reindex(df.index)

# Detect new / deleted posts
prev_links = set(df_prev.index)

new_posts = ~df.index.isin(prev_links)
print("Posts changed from", len(df_prev), "to", len(df))
print('New posts', new_posts.sum())
print('Deleted posts', len(df_prev) - (len(df) - new_posts.sum()))
df.loc[new_posts, "Posted After"] = (prev_date + timedelta(days=1)).isoformat()
df.loc[new_posts, "Posted By"] = date.today().isoformat()

# Restore URL as a regular column and match df_prev column order
df = df.reset_index()
df_prev = df_prev.reset_index()
# df_prev columns first (preserving order), then any new columns from df
col_order = [c for c in df_prev.columns if c in df.columns] + [c for c in df.columns if c not in df_prev.columns]
df = df[col_order]

# Sort by Rank
if "Rank" in df.columns:
    df = df.sort_values("Rank", ascending=False).reset_index(drop=True)

Most recent prior sheet: '2026-02-11'
Posts changed from 175 to 175
New posts 6
Deleted posts 6


In [10]:
import csv
from io import StringIO

df.to_csv(f"workspace/scrape/jobs/waymo/data/waymo_jobs_{date.today().isoformat()}.csv", index=False)
csv_path = f"workspace/scrape/jobs/waymo/data/waymo_jobs_{date.today().isoformat()}.csv"
sheet_name = date.today().isoformat()

existing = {w.title for w in sh.worksheets()}
if sheet_name in existing:
    n = 2
    while f"{sheet_name}.{n}" in existing:
        n += 1
    sheet_name = f"{sheet_name}.{n}"

ws = sh.add_worksheet(title=sheet_name, rows=len(df) + 1, cols=len(df.columns))

csv_data = open(csv_path, "r", encoding="utf-8").read()
rows = list(csv.reader(StringIO(csv_data)))
ws.update(rows, value_input_option="USER_ENTERED")
print(f"Uploaded {len(rows)-1} rows to new sheet '{sheet_name}'")

Uploaded 175 rows to new sheet '2026-02-13.2'
